---
title: "2023_Kronensicherung_Bosau_Baumdaten"
author: "Kyell Jensen"
date: "2025-10-02"
format: pdf
editor: visual
---

%load_ext autoreload
%autoreload 2

%aimport arbolab

# 2023_Kronensicherung_Bosau_Baumdaten

## Analyse der TreeMotionSensor-Daten

Nutze eine geeignete Python 3.12 Umgebung (z. B. virtuelle Environment) und installiere das Paket arbolab inklusive treemtion and windy plugin.

## Package Importe

Es werden zuerst benötigte Standard-Pakete importiert. Nachfolgend wird das extra erstellte arbolab Paket importiert. Fehler beim Import dieses Pakets sind ggf. Bugs. Das Paket bietet eine gemeinsame Code Basis für die plugins treemotion und windy, die Geräte spezifische logik mitbringen.

### Public imports


In [1]:
from pathlib import Path
from datetime import datetime, time

import pandas as pd

## Import Arbolab-Package

Das Paket Arbolab wurde vom Autor (Kyell Jensen) zum einfachen Analysieren, Plotten und zur Interpretation der Messdaten verschiedener Messgeräte im Kontext Arboristik geschrieben. Das Plugin treemotion bietet Support für TreeMotionSensoren (Accelerometer/Inclinometer) der Firma "IML Instrumenta Mechanik Labor Electronic" (ehemals "Argus Electronic") (https://www.iml-electronic.de/produkt/picus-treemotion/). Das Paket inklusive plugins ist auf GitHub verfügbar (https://github.com/Kyellsen/arbolab). Die Messdaten der Geräte müssen für die Verarbeitung in Python erst in die Firmen eigene Software "TMS.Software" importiert werden. Von dort ist ein manueller Export in einem lesbaren CSV-Format möglich. Dieser Schritt lässt sich nicht automatisieren.

In [ ]:
from arbolab import Lab, LoggerConfig, configure_logger, get_logger, tms, windy
from arbolab.models import Project, Experiment, Measurement, Tree, TreatmentCable

## Daten Importe

Lege Pfade für Daten-Importe, Daten-Exporte etc. fest (ggf. anpassen an eigene Verzeichnisstruktur)

In [3]:
# Main
analyse_name = r"2022_Kronensicherung_Bosau_2025-10-02"
workspace_path = Path(r"C:\kyellsen\005_Projekte\2024_BA\031_Feldversuch_2023_Bosau\030_Analysen\2022_Kronensicherung_Bosau_Schwingungen\working_dir")
input_path = Path(r"C:\kyellsen\005_Projekte\2022_Bosau")
tms_data_input_dir = input_path / "021_Daten_Test" / "TMS"
tms_data_input_dir_e1 = tms_data_input_dir / Path("CSV_Messung_001_export_2022-01-29_24h")

### Lade der Versuchsdatenbank

Die Versuchsdaten wurden im Feld in Google Sheets Tabellen dokumentiert und dann in eine DuckDB überführt. DuckDB ist ähnlich SQlite eine lokal lauffähige relationale Datenbank, ist allerdings für moderne spaltenbasiert Data Science Workflows optimiert.
Die Datenbank ist die zentrale stelle für alle Metadaten des Feldversuchs in Bosau.
Änderungen an der Ausgangs-Datenbank sollen vermieden werden - sie wird entsprechend einmalig ins 'working_dir' kopiert. Die weitere Analyse erfolgt dann auf der Kopie der Datenbank

In [4]:
from pathlib import Path
import shutil

# Falls die Datenbank "mydata.duckdb" heißt:
src_file = Path(r"C:\kyellsen\006_Packages\arbolab\examples\test_workspaces\treemotion_sqlite_lab_migration\arbolab.duckdb")
dst_file = workspace_path / "arbolab.duckdb"


# Datei kopieren (inkl. Metadaten)
shutil.copy2(src_file, dst_file)

print(f"Datenbank kopiert \n      von: {src_file} \n      nach: {dst_file}")

Datenbank kopiert 
      von: C:\kyellsen\006_Packages\arbolab\examples\test_workspaces\treemotion_sqlite_lab_migration\arbolab.duckdb 
      nach: C:\kyellsen\005_Projekte\2024_BA\031_Feldversuch_2023_Bosau\030_Analysen\2022_Kronensicherung_Bosau_Schwingungen\working_dir\arbolab.duckdb


### Initialisieren des arbolab Lab´s

Für die Funktionalität des arbolab Paketes wird ein Lab erstellt, in dem weitere Managerinstanzen etc. bereit stehen. Es wird eine Verbindung zur bereits bestehenden Datenbank hergestellt.

In [5]:
configure_logger(LoggerConfig(level="INFO"))
logger = get_logger("arbolab.examples")

lab = Lab.setup(workspace_path=workspace_path)

plugin_names = lab.list_plugins()
summary = ", ".join(plugin_names) if plugin_names else "<none>"
logger.info("Registered plugins: %s", summary)
plugin = tms.get_plugin(lab)
logger.info("TreeMotion plugin metadata: %s", plugin.get_metadata())

windy_plugin = windy.get_plugin(lab)
logger.info("Windy plugin metadata: %s", windy_plugin.get_metadata())

2025-10-02 17:15:45 | arbolab | INFO | __init__ | DeviceRegistry initialised
2025-10-02 17:15:45 | arbolab | INFO | __init__ | WorkspacePluginManager initialised
2025-10-02 17:15:45 | arbolab | WARNING | setup | Workspace already exists at C:\kyellsen\005_Projekte\2024_BA\031_Feldversuch_2023_Bosau\030_Analysen\2022_Kronensicherung_Bosau_Schwingungen\working_dir; loading existing workspace instead
2025-10-02 17:15:45 | arbolab | INFO | load | Loading workspace from C:\kyellsen\005_Projekte\2024_BA\031_Feldversuch_2023_Bosau\030_Analysen\2022_Kronensicherung_Bosau_Schwingungen\working_dir
2025-10-02 17:15:45 | arbolab | INFO | __init__ | DatabaseManager initialised with database file C:\kyellsen\005_Projekte\2024_BA\031_Feldversuch_2023_Bosau\030_Analysen\2022_Kronensicherung_Bosau_Schwingungen\working_dir\arbolab.duckdb
2025-10-02 17:15:45 | arbolab | INFO | __init__ | StorageManager initialised for workspace C:\kyellsen\005_Projekte\2024_BA\031_Feldversuch_2023_Bosau\030_Analysen\2022

### Metadaten zum Versuch laden (Sensorpositionierung, Versuchsprotokoll/Messablauf, Baumdaten)

Es folgt die Verbindung mit der Datenbank und das Laden der 'Project'-Instance. Innerhalb des Projektes können dann bestimmte Messreihen ('Experiment') und Messungen ('Measurement') geladen werden.

In [6]:
with lab.session_scope() as session:
    p1 = session.get(Project, 1)
    e1 = session.get(Experiment, 1)
    m1 = session.get(Measurement, 1)

    if p1 is None:
        raise RuntimeError("Project with id 1 not found in the database")
    if e1 is None:
        raise RuntimeError("Experiment with id 1 not found in the database")
    if m1 is None:
        raise RuntimeError("Experiment with id 1 not found in the database")
    
    print(p1)
    print(e1)
    print(m1)

Project(id=1, name='Bosau TreeMotion Migration', namespace='arbolab')
Experiment(id=1, name='Messreihe 1', namespace='arbolab', project_id=1, valid_from=datetime.datetime(2022, 1, 23, 1, 0, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>), valid_until=datetime.datetime(2022, 2, 1, 1, 0, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>))
Measurement(id=1, name='Measurement 1', namespace='arbolab', trial_id=1, sensor_id=22, placement_id=1)


Die Messdaten der TreeMotionSensoren wurden von den Geräten gesichert und aus der "TMS.Software" als CSV-Dateien exportiert. Die spätere Zuordnung erfolgt über die Messreihe (Experiment) und die Sensor ID. Hierzu steht im tms Plugin die Methode 'tms.find_files' zur Verfügung. Dazu wird der Pfad übergeben, indem alle CSV-Dateien einer Messreihe liegen. Die letzten stellen des Dateinames ermöglichen eine Zuordnung zum Sensor, der bereits über Measurement in der Datenbank hinterlegt ist. Bedingung: Je Messreihe darf nur eine Messdatei je Sensor vorliegen!

Die Funktion wird nachfolgend für die Messreihe 1 aufgerufen. 


In [7]:
with lab.session_scope() as session:
    experiment = session.get(Experiment, 1)
    if experiment is None:
        raise RuntimeError("Experiment with id 1 not found in the database")

    matches = tms.find_files(
        lab=lab,
        target=experiment,
        data_directory=tms_data_input_dir_e1,
        session=session,
        auto_commit=True,
    )
    logger.info("Resolved TreeMotion CSV files for %d measurement(s)", len(matches))

2025-10-02 17:15:48 | arbolab_plugins.treemotion.services.file_discovery | INFO | find_files | Starting TreeMotion file discovery in 'C:\kyellsen\005_Projekte\2022_Bosau\021_Daten_Test\TMS\CSV_Messung_001_export_2022-01-29_24h'.
2025-10-02 17:15:48 | arbolab_plugins.treemotion.services.file_discovery | INFO | find_files | Matched measurement 1 (TreeMotion sensor 76) to file '2022-01-29 020000__DatasTI0000000076.csv'.
2025-10-02 17:15:48 | arbolab_plugins.treemotion.services.file_discovery | WARNING | find_files | No CSV file found for measurement 2 (TreeMotion sensor 75) in 'CSV_Messung_001_export_2022-01-29_24h'.
2025-10-02 17:15:48 | arbolab_plugins.treemotion.services.file_discovery | WARNING | find_files | No CSV file found for measurement 3 (TreeMotion sensor 27) in 'CSV_Messung_001_export_2022-01-29_24h'.
2025-10-02 17:15:48 | arbolab_plugins.treemotion.services.file_discovery | WARNING | find_files | No CSV file found for measurement 4 (TreeMotion sensor 77) in 'CSV_Messung_001_

Die Funktion 'load_from_csv' liest dann die CSV-Daten ein und erstellt eine Variante "raw" der Daten der Messung. Eine Messung kann entsprechend in verschiedenen Verarbeitungszuständen abgespeichert werden. Die Messdaten werden nicht in der Datenbank, sondern als .parquet-Dateien mit Referenz in der Datenbank gespeichert. Diese ermöglicht ein effizienteres Lesen als CSV und zugleich Abfragen mit SQL über die DuckDB

In [ ]:
with lab.session_scope() as session:
    experiment = session.get(Experiment, 1)
    if experiment is None:
        raise RuntimeError("Experiment with id 1 not found in the database")
    tms_variants = tms.load_from_csv(lab=lab,
        target=experiment,
        update_existing=True,
        session=session,
        auto_commit=True,
    )

2025-10-02 17:15:49 | arbolab.tms | INFO | load_from_csv | Stored TreeMotion dataset for measurement 1 as variant 'raw' (1728002 rows).
2025-10-02 17:15:49 | arbolab.tms | WARNING | load_from_csv | Measurement 2 lacks TreeMotion CSV provenance metadata; skipping.
2025-10-02 17:15:49 | arbolab.tms | WARNING | load_from_csv | Measurement 3 lacks TreeMotion CSV provenance metadata; skipping.
2025-10-02 17:15:49 | arbolab.tms | WARNING | load_from_csv | Measurement 4 lacks TreeMotion CSV provenance metadata; skipping.
2025-10-02 17:15:50 | arbolab.tms | INFO | load_from_csv | Stored TreeMotion dataset for measurement 5 as variant 'raw' (1651274 rows).
2025-10-02 17:15:50 | arbolab.tms | WARNING | load_from_csv | Measurement 6 lacks TreeMotion CSV provenance metadata; skipping.
2025-10-02 17:15:50 | arbolab.tms | WARNING | load_from_csv | Measurement 7 lacks TreeMotion CSV provenance metadata; skipping.
2025-10-02 17:15:50 | arbolab.tms | WARNING | load_from_csv | Measurement 8 lacks TreeMo

In [16]:
with lab.session_scope() as session:

    # Abfragen aller Daten aus der Tabelle 'Tree'
    tree_data = session.query(Tree).all()

# Erstellen eines DataFrames aus den abgefragten Daten für 'Tree'
tree_df = pd.DataFrame([{
    'Umfang': tree.circumference,
    'Höhe (ca.)': tree.height,
    'Vergabelungshöhe': tree.fork_height
} for tree in tree_data])

tree_df

,Umfang,Höhe (ca.),Vergabelungshöhe
0,NaN,2200.0,NaN
1,NaN,NaN,NaN
2,239.0,2200.0,353.0
3,229.0,2200.0,384.0
4,217.0,2200.0,346.0
5,NaN,NaN,NaN
6,233.0,2200.0,422.0
7,246.0,2200.0,384.0
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [21]:
with lab.session_scope() as session:

    # Abfragen aller Daten aus der Tabelle 'TreatmentCable'
    treatment_cable = session.query(TreatmentCable).all()

# Erstellen eines DataFrames aus den abgefragten Daten für 'TreeCable'
treatment_cable_df = pd.DataFrame([{
    'Höhe KS': cable.height,
    'Länge KS': cable.length,
    'Umfang Stämmlinge A auf Höhe KS': cable.trunk_circumference_a,
    'Umfang Stämmlinge B auf Höhe KS': cable.trunk_circumference_b
} for cable in treatment_cable])

# Kombinieren der Umfänge von Stämmen A und B in einer gemeinsamen Serie
combined_circumference = pd.concat([
    treatment_cable_df['Umfang Stämmlinge A auf Höhe KS'], 
    treatment_cable_df['Umfang Stämmlinge B auf Höhe KS']
])

metrics = ['min', 'mean', 'max', 'std']
treatment_cable_df
combined_circumference

0     52.0
1     51.0
2     51.0
3     63.0
4     57.0
5      NaN
6     57.0
7     57.0
8     50.0
9     50.0
10    60.0
11    52.0
0     57.0
1     58.0
2     47.0
3     47.0
4     67.0
5      NaN
6     67.0
7     67.0
8     55.0
9     50.0
10    70.0
11    51.0
dtype: float64

In [24]:


# Statistische Auswertungen für die kombinierten Umfänge
circumference_statistics = combined_circumference.agg(metrics)
circumference_statistics = pd.DataFrame(circumference_statistics).transpose()
circumference_statistics.index = ['Umfang Stämmlinge auf Höhe KS']

# Statistische Auswertungen für die Spalten in 'Tree' DataFrame
tree_statistics = tree_df.agg(metrics).transpose()

# Statistische Auswertungen für die anderen Spalten in 'TreeCable' DataFrame
treatment_cable_df_statistics = treatment_cable_df[['Höhe KS', 'Länge KS']].agg(metrics).transpose()

# Zusammenführen der Statistiken in einem DataFrame
combined_statistics = pd.concat([tree_statistics, treatment_cable_df_statistics, circumference_statistics])
combined_statistics["Einheit"] = "cm"

combined_statistics

,min,mean,max,std,Einheit
Umfang,217.0,234.900000,258.0,11.376877,cm
Höhe (ca.),2200.0,2200.000000,2200.0,0.000000,cm
Vergabelungshöhe,316.0,387.700000,510.0,55.643808,cm
Höhe KS,1481.0,1611.636364,1746.0,100.477134,cm
Länge KS,136.0,218.090909,290.0,41.810177,cm
Umfang Stämmlinge auf Höhe KS,47.0,56.181818,70.0,6.932576,cm


In [26]:
# Umwandlung der statistischen Auswertung in LaTeX
latex_string = combined_statistics.to_latex(index=True, escape=True, column_format="lrrrrr", header=["Min.", "Mean", "Max.", "Std.", "Einheit"], 
                                            float_format="{:0.2f}".format)

# LaTeX Tabellen-String formatieren
latex_table = f"""
\\begin{{table}}[h]
    \\centering
    {latex_string}
    \\caption{{Plesse - Daten der Versuchsbäume}}
    \\label{{tab:plesse_versuchsbaeume}}
\\end{{table}}
"""

print(latex_table)


\begin{table}[h]
    \centering
    \begin{tabular}{lrrrrr}
\toprule
 & Min. & Mean & Max. & Std. & Einheit \\
\midrule
Umfang & 217.00 & 234.90 & 258.00 & 11.38 & cm \\
Höhe (ca.) & 2200.00 & 2200.00 & 2200.00 & 0.00 & cm \\
Vergabelungshöhe & 316.00 & 387.70 & 510.00 & 55.64 & cm \\
Höhe KS & 1481.00 & 1611.64 & 1746.00 & 100.48 & cm \\
Länge KS & 136.00 & 218.09 & 290.00 & 41.81 & cm \\
Umfang Stämmlinge auf Höhe KS & 47.00 & 56.18 & 70.00 & 6.93 & cm \\
\bottomrule
\end{tabular}

    \caption{Plesse - Daten der Versuchsbäume}
    \label{tab:plesse_versuchsbaeume}
\end{table}



In [ ]:
    
with lab.session_scope() as session:
    p1 = session.get(Project, 1)  
    if p1 is None:
        raise RuntimeError("Project with id 1 not found in the database")  
    wind_variants = windy.load_station_measurement(
        lab=lab,
        target=p1,
        station_id="06163",
        measurement_name="DWD Wind Station 06163 Dörnick",
        start_date=datetime(2020, 1, 1),
        end_date=datetime(2023, 1, 31),
        session=session,
        auto_commit=True,
    )
    logger.info("Loaded wind data with %d variant(s)", len(wind_variants))
    

2025-10-02 17:49:07 | arbolab.windy | INFO | _log_station_request | Loading Windy station measurement: project_id=1 station_id=06163 target_variant=raw src_variant=<auto> update_existing=True start_date=2020-01-01T00:00:00 end_date=2023-01-31T00:00:00 periods=<default> parameters=<all>
2025-10-02 17:49:07 | arbolab_plugins.windy.services.wetterdienst | INFO | load_station_dataset | Requesting Wetterdienst wind data for station 06163
2025-10-02 17:49:10 | arbolab_plugins.windy.services.wetterdienst | INFO | load_station_dataset | Received 324290 rows (14 columns) for station 06163 from Wetterdienst in 3.82s
2025-10-02 17:49:10 | arbolab.windy | INFO | _load_windy_dataset | Retrieved Wetterdienst wind data for station 06163 (rows=324290)
2025-10-02 17:49:10 | arbolab.models.measurement.Measurement | INFO | _log_creation | Instantiated Measurement(name='DWD Wind Station 06163 Dörnick', project_id=1, channel='06163')
2025-10-02 17:49:10 | arbolab.windy | INFO | _ensure_measurement | Create

{130: DataVariant(id=4, namespace='arbolab', measurement_id=130, variant_name='raw', data_format='parquet')}
